In [1]:
import pandas as pd
import numpy as np
import os
import re
import datetime
import time

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools
init_notebook_mode(connected=True)

import matplotlib.pyplot as plt
%matplotlib inline

import progressbar

import seaborn as sns

In [14]:
time_zone_interval = 12

In [3]:
#read in files' names
path = '/Volumes/Mac/School/Graduate-Columbia/Business_Analytics_IEOR4650/Project/Data/'  #please use relative path for team's convience
file_names = list()
for path, subdirs, files in os.walk(path):  #read root and subfolder recursively
    for filename in files:
        if re.search(pattern=r'csv', string=filename):  #store only csv
            # contain both path and file name eg)  './Data/Weather/nyc_weahter.csv
            file_names.append(os.path.join(path, filename))
        else:
            continue
            
file_names

['/Volumes/Mac/School/Graduate-Columbia/Business_Analytics_IEOR4650/Project/Data/fhv_tripdata_2018-01.csv',
 '/Volumes/Mac/School/Graduate-Columbia/Business_Analytics_IEOR4650/Project/Data/fhv_tripdata_2018-02.csv',
 '/Volumes/Mac/School/Graduate-Columbia/Business_Analytics_IEOR4650/Project/Data/fhv_tripdata_2018-03.csv',
 '/Volumes/Mac/School/Graduate-Columbia/Business_Analytics_IEOR4650/Project/Data/fhv_tripdata_2018-04.csv',
 '/Volumes/Mac/School/Graduate-Columbia/Business_Analytics_IEOR4650/Project/Data/fhv_tripdata_2018-05.csv',
 '/Volumes/Mac/School/Graduate-Columbia/Business_Analytics_IEOR4650/Project/Data/fhv_tripdata_2018-06.csv',
 '/Volumes/Mac/School/Graduate-Columbia/Business_Analytics_IEOR4650/Project/Data/fhv_tripdata_2018-07.csv',
 '/Volumes/Mac/School/Graduate-Columbia/Business_Analytics_IEOR4650/Project/Data/fhv_tripdata_2018-08.csv',
 '/Volumes/Mac/School/Graduate-Columbia/Business_Analytics_IEOR4650/Project/Data/fhv_tripdata_2018-09.csv',
 '/Volumes/Mac/School/Gradua

In [4]:
#### No need to rerun  ####
## Sampling the data to 1/12 of size
sample_frac = 1
beg_time = time.time()
yellow_csv = []
green_csv = []
fhv_csv = []
additional_features_file_names = []
count = 0
total = len(file_names)
bar = progressbar.ProgressBar(max_value=total)

for file_name in file_names:
    count = count + 1
    bar.update(count)# print out the progress and running time
    
    if re.search(pattern=r'yellow', string=file_name):
        df = pd.read_csv(file_name)
        df=df.sample(frac=sample_frac, random_state=1)
        yellow_csv.append(df)
    else:
        time.sleep(0.0001)
        #not yet loaded into dataframe need to merge manually
        additional_features_file_names.append(file_name)
        
        
additional_features_file_names


 88% (37 of 42) |#####################   | Elapsed Time: 0:05:53 ETA:   0:03:09

['/Volumes/Mac/School/Graduate-Columbia/Business_Analytics_IEOR4650/Project/Data/fhv_tripdata_2018-01.csv',
 '/Volumes/Mac/School/Graduate-Columbia/Business_Analytics_IEOR4650/Project/Data/fhv_tripdata_2018-02.csv',
 '/Volumes/Mac/School/Graduate-Columbia/Business_Analytics_IEOR4650/Project/Data/fhv_tripdata_2018-03.csv',
 '/Volumes/Mac/School/Graduate-Columbia/Business_Analytics_IEOR4650/Project/Data/fhv_tripdata_2018-04.csv',
 '/Volumes/Mac/School/Graduate-Columbia/Business_Analytics_IEOR4650/Project/Data/fhv_tripdata_2018-05.csv',
 '/Volumes/Mac/School/Graduate-Columbia/Business_Analytics_IEOR4650/Project/Data/fhv_tripdata_2018-06.csv',
 '/Volumes/Mac/School/Graduate-Columbia/Business_Analytics_IEOR4650/Project/Data/fhv_tripdata_2018-07.csv',
 '/Volumes/Mac/School/Graduate-Columbia/Business_Analytics_IEOR4650/Project/Data/fhv_tripdata_2018-08.csv',
 '/Volumes/Mac/School/Graduate-Columbia/Business_Analytics_IEOR4650/Project/Data/fhv_tripdata_2018-09.csv',
 '/Volumes/Mac/School/Gradua

In [5]:
## concat the monthly data into one file for each vehicle type
df_yellow = pd.concat(yellow_csv, axis=0)

# add a column of vehicle_type to each df 
df_yellow['vehicle_type'] = 'yellow'

## save above process to a single csv for green and yellow data
df_yellow.to_csv('yellow.csv')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




In [42]:
df_yellow.to_csv('/Volumes/Mac/School/Graduate-Columbia/Business_Analytics_IEOR4650/Project/Data/full_yellow.csv')

In [10]:
yellow = df_yellow
yellow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97631424 entries, 3465173 to 4322341
Data columns (total 19 columns):
DOLocationID             int64
PULocationID             int64
RatecodeID               int64
Unnamed: 0               float64
VendorID                 int64
extra                    float64
fare_amount              float64
improvement_surcharge    float64
mta_tax                  float64
passenger_count          int64
payment_type             int64
store_and_fwd_flag       object
tip_amount               float64
tolls_amount             float64
total_amount             float64
tpep_dropoff_datetime    object
tpep_pickup_datetime     object
trip_distance            float64
vehicle_type             object
dtypes: float64(9), int64(6), object(4)
memory usage: 14.5+ GB


In [12]:
yellow = df_yellow[['DOLocationID',
                    'PULocationID',
                    'tpep_dropoff_datetime',
                   'tpep_pickup_datetime']]
yellow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97631424 entries, 3465173 to 4322341
Data columns (total 4 columns):
DOLocationID             int64
PULocationID             int64
tpep_dropoff_datetime    object
tpep_pickup_datetime     object
dtypes: int64(2), object(2)
memory usage: 3.6+ GB


In [28]:
# create timezone with interval 15min
yellow['Time_Diff'] = pd.to_datetime(yellow['tpep_dropoff_datetime']) - pd.to_datetime(yellow['tpep_pickup_datetime'])
yellow.shape
yellow['Time_Diff'].mean()
yellow['dropoff_datetime'] = yellow['tpep_dropoff_datetime'].str[11:13].astype(float) * 60 + yellow['tpep_dropoff_datetime'].str[14:16].astype(float) + yellow['tpep_dropoff_datetime'].str[17:19].astype(float) / 60
yellow['pickup_datetime'] = yellow['tpep_pickup_datetime'].str[11:13].astype(float) * 60 + yellow['tpep_pickup_datetime'].str[14:16].astype(float) + yellow['tpep_pickup_datetime'].str[17:19].astype(float) / 60
yellow['dropoff_timezone'] = (yellow['dropoff_datetime'] //time_zone_interval)
yellow['pickup_timezone'] = (yellow['pickup_datetime'] // time_zone_interval)

## clean mistakes
yellow['pickup_date'] = pd.to_datetime(yellow['tpep_pickup_datetime']).dt.date
yellow['dropoff_date'] = pd.to_datetime(yellow['tpep_dropoff_datetime']).dt.date
yellow = yellow[yellow['pickup_date']==yellow['dropoff_date']]

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [34]:
yellow_new = yellow[['PULocationID','DOLocationID','Time_Diff',]]
yellow_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92352234 entries, 4463675 to 4322341
Data columns (total 3 columns):
PULocationID    int64
DOLocationID    int64
Time_Diff       timedelta64[ns]
dtypes: int64(2), timedelta64[ns](1)
memory usage: 2.8 GB


In [35]:
%%time
td = yellow_new[['PULocationID','DOLocationID','Time_Diff']]
td['Time_Diff_Min'] = td['Time_Diff'] / np.timedelta64(1,'m')
td2 = td.groupby(['PULocationID','DOLocationID']).mean()

index = list(range(266))
columns = list(range(266))
zone_distance_matrix = pd.DataFrame(index=index, columns=columns)

for ind in td2.index:
    PU_id = ind[0]
    DO_id = ind[1]
    zone_distance_matrix.loc[PU_id,DO_id] = td2.loc[PU_id,DO_id][0]


CPU times: user 2min 4s, sys: 12.4 s, total: 2min 17s
Wall time: 2min 19s


In [36]:
zone_distance_matrix

,0,1,2,3,4,5,6,7,8,9,...,256,257,258,259,260,261,262,263,264,265
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0.736837,NaN,NaN,22.5417,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,36.8667,NaN,NaN,0.212048,22.0425
2,NaN,189.467,8.67381,NaN,NaN,NaN,NaN,39.5333,NaN,29.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.5333,NaN,9.61667
3,NaN,NaN,NaN,5.11791,39.3667,NaN,NaN,41.9218,NaN,NaN,...,NaN,NaN,NaN,15.9773,NaN,NaN,NaN,NaN,NaN,16.5437
4,NaN,43.6141,NaN,36.8767,4.23341,38.4667,36.215,24.5779,27.7167,25.9333,...,14.067,24.2884,40.4744,33.5722,25.2034,13.9331,17.3754,18.1467,4.66232,37.8282
5,NaN,NaN,NaN,NaN,NaN,1.77677,25.9333,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,15.2667
6,NaN,31.25,NaN,NaN,6.71667,NaN,1.49455,NaN,NaN,NaN,...,NaN,20.8,NaN,NaN,NaN,29.3167,NaN,NaN,NaN,54.5167
7,NaN,49.0508,58.8667,33.0278,22.8655,59.8,27.45,4.85029,8.11711,26.8894,...,21.174,33.2053,28.5677,31.3115,9.24898,32.0841,19.0024,17.8838,4.75859,38.4565
8,NaN,67.5233,NaN,NaN,NaN,NaN,NaN,8.42679,10.4616,NaN,...,NaN,NaN,NaN,NaN,14.3407,NaN,14.0474,12.3192,0,40.7388
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.1233,NaN,3.43746,...,NaN,47.5417,30.2369,NaN,22.8292,NaN,31.0833,NaN,8.66667,30.7074


In [37]:
sum(len(zone_distance_matrix) - zone_distance_matrix.count())


22808

In [38]:
zone_distance_matrix.to_csv('./Data/zone_distance_matrix.csv')

In [39]:
zone_distance_matrix.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 266 entries, 0 to 265
Columns: 266 entries, 0 to 265
dtypes: object(266)
memory usage: 564.9+ KB


In [1]:
## loading the distance matrix(compute from the full_yellow.csv)
M = 1000000000000 # large number
zone_dis = zone_distance_matrix
zone_dis = zone_dis.fillna(M)
arr_zone_dis = np.array(zone_dis)
zone_dis_M = np.where(arr_zone_dis==M ,arr_zone_dis.T,arr_zone_dis)
# zone_dis_M = np.where(zone_dis_M==M ,np.nan ,zone_dis_M)

NameError: name 'zone_distance_matrix' is not defined

In [ ]:
# create graph and prepare to fill the nan
G=nx.Graph()
rows = zone_dis_M.shape[0]
cols = zone_dis_M.shape[1]
for i in range(rows):
    for j in range(cols):
        G.add_edge(i,j, weight=zone_dis_M[i,j])

In [ ]:
%%time
## complete the incomplete distance matrix by triangle inequality
bar = progressbar.ProgressBar(max_value=rows-1)

for i in range(rows):
    bar.update(i)# print out the progress and running time

    for j in range(cols):
        if zone_dis_M[i,j] == M:
            shortest_distance = nx.dijkstra_path_length(G,i,j)
            if shortest_distance < M:
                zone_dis_M[i,j] = shortest_distance

In [ ]:
## Saving 
# zone_dis_M = zone_dis_M[1:,1:]
zone_dis_M = np.where(zone_dis_M==M ,np.nan ,zone_dis_M)
pd.DataFrame(zone_dis_M).to_csv('./Data/full_zone_distance_matrix.csv')